In [2]:
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer=LancasterStemmer()

import tensorflow as tf
import numpy as np
import tflearn
import random
import json

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rissu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
file=open('intents.json')
data=json.load(file)

In [21]:
words=[]
classes=[]
docs=[]
ignore=['?']

for i in data['intents']:
    for p in i['patterns']:
        w=nltk.word_tokenize(p)
        words.extend(w)
        docs.append((w,i['tag']))
    classes.append(i['tag'])

In [25]:
words=[stemmer.stem(w.lower()) for w in words if w not in ignore]

In [31]:
words=sorted(list(set(words)))

In [117]:
training=[]
for doc in docs:
    temp=[]
    b=[0]*len(classes)
    pattern_word=doc[0]
    pattern_word=[stemmer.stem(w.lower()) for w in pattern_word]
    for w in words:
        temp.append(1) if w in pattern_word else temp.append(0)
    b[classes.index(doc[1])]=1
    training.append([temp,b])
random.shuffle(training)
training=np.array(training)
train_X=list(training[:,0])
train_y=list(training[:,1])

In [130]:
tf.reset_default_graph()
net=tflearn.input_data(shape=[None,len(train_X[0])])
net=tflearn.fully_connected(net,10)
net=tflearn.fully_connected(net,10)
net=tflearn.fully_connected(net,len(train_y[0]),activation='softmax')
net=tflearn.regression(net)

model=tflearn.DNN(net,tensorboard_dir='tflearn_logs')

model.fit(train_X,train_y,n_epoch=1000,batch_size=8,show_metric=True)

Training Step: 3999  | total loss: 0.30289 | time: 0.007s
| Adam | epoch: 1000 | loss: 0.30289 - acc: 0.9838 -- iter: 24/31
Training Step: 4000  | total loss: 0.27305 | time: 0.008s
| Adam | epoch: 1000 | loss: 0.27305 - acc: 0.9854 -- iter: 31/31
--


In [132]:
import pickle
pickle.dump({'words':words,
             'classes':classes,
             'train_x':train_X,
             'train_y':train_y},open('training_data','wb'))

In [133]:
data=pickle.load(open('training_data','rb'))

In [151]:
words=data['words']
classes=data['classes']
train_x=data['train_x']
train_y=data['train_y']

In [138]:
intents=json.load(open('intents.json'))

In [146]:
def clean_sentence(sentence):
    words=nltk.word_tokenize(sentence)
    stemd=[stemmer.stem(w.lower()) for w in words]
    return stemd
clean_sentence('I am sitting in the floor')
    

['i', 'am', 'sit', 'in', 'the', 'flo']

In [159]:
def bagofwords(sentence,words):
    word=clean_sentence(sentence)
    bow=[]
    for i in words:
        if i in word:
            bow.append(1)
        else:
            bow.append(0)
    return np.array(bow)
bagofwords('My name is Rishav,',words)


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [178]:
THRESOLD=0.3
def classify(sentence):
    a=[]
    result=bagofwords(sentence,words)
    predicted=model.predict([result])[0]
    for i,j in zip(predicted,classes):
        if i>=THRESOLD:
            a.append([i,j])
    a.sort(key=lambda x: x[1] ,reverse=False)
    a=a[0]
    return (a)
a=classify('My name is Rishav,')
a

[0.570622, 'greeting']

In [241]:
def response(sentence):
    a=classify(sentence)[1]
    intt=intents['intents']
    for i in intt:
        if i['tag']==a:
            respons=i['responses']
            return random.choice(respons)

In [305]:
response('What is your location?')

'We are on the intersection of London Alley and Bridge Avenue.'

In [293]:
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day'],
   'responses': ['Hello, thanks for visiting',
    'Good to see you again',
    'Hi there, how can I help?'],
   'context_set': ''},
  {'tag': 'goodbye',
   'patterns': ['Bye', 'See you later', 'Goodbye'],
   'responses': ['See you later, thanks for visiting',
    'Have a nice day',
    'Bye! Come back again soon.']},
  {'tag': 'thanks',
   'patterns': ['Thanks', 'Thank you', "That's helpful"],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure']},
  {'tag': 'hours',
   'patterns': ['What hours are you open?',
    'What are your hours?',
    'When are you open?'],
   'responses': ["We're open every day 9am-9pm",
    'Our hours are 9am-9pm every day']},
  {'tag': 'location',
   'patterns': ['What is your location?',
    'Where are you located?',
    'What is your address?',
    'Where is your restaurant situated?'],
   'responses': ['We are on the intersection of Lon